In [ ]:
#TODO: increase size
#TODO: change train to include labels, ignore for ae

In [ ]:
from datetime import datetime
import tensorflow as tf
import keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Layer, Dense, Conv2D, Conv2DTranspose , LeakyReLU, Dropout, Flatten, Input, Reshape

In [ ]:
#run for no gpu if needed
tf.config.set_visible_devices([], 'GPU')

DATA_DIR='/home/evan/Datasets/tensorflow'
BATCH_SIZE=32
BUFFER_SIZE = 10000
IMG_SIZE = (64,64)
EMBEDDED_DIM = 32

train_dataset = tfds.load('stanford_dogs', as_supervised=True, split='train', data_dir=DATA_DIR, download=False)
test_dataset = tfds.load('stanford_dogs', as_supervised=True, split='test', data_dir=DATA_DIR, download=False)

def transform(x , y):
    x = tf.image.resize(x, IMG_SIZE)
    x = tf.cast(x, tf.float32)
    x = (x - 127.5)/127.5
    return x, y
    
# train_dataset = train_dataset.map(lambda x, y: (tf.cast(x, tf.uint8), y))

def transform_ae(x,y):
    return (x,x,y)

def prepare_transformed(dataset):
    return dataset.repeat().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

test_dataset = test_dataset.map(lambda x, y : transform(x,y))
train_dataset = train_dataset.map(lambda x, y : transform(x,y))
ae_dataset = train_dataset.map(lambda x, y : transform_ae(x,y))
ae_dataset = prepare_transformed(ae_dataset)
train_dataset = prepare_transformed(train_dataset)

print(train_dataset)
print(ae_dataset)
print(test_dataset)

In [ ]:
class Conv2DTransposeTied(Layer):
    def __init__(self, conv2D, activation=None, **kwargs):
        self.conv2D = conv2D
        self.activation = keras.activations.get(activation)
        super().__init__(**kwargs)
    
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name='bias', initializer ='zeros',
                                      shape= self.conv2D.input_shape[1:]) #exclude batch dim
    
    def call(self, inputs):
        print(inputs)
        print(self.conv2D.input_shape)
        Z = tf.nn.conv2d_transpose(inputs, self.conv2D.kernel, (BATCH_SIZE,) + self.conv2D.input_shape[1:], self.conv2D.strides, padding=self.conv2D.padding.upper())
        return self.activation(Z + self.biases)

In [ ]:
if load_ae = True:
    autoencoder = keras.models.load_model('/home/evan/Desktop/Convolutional Random Forest/autoencoder')

In [ ]:
class Autoencoder(Model):
    def __init__(self):
        super().__init__()
        self.encoder = self.create_encoder()
        self.decoder = self.create_decoder()
        self.autoencoder = Sequential([self.encoder, self.decoder])

    def create_encoder(self):
        encoder_input_img = Input(IMG_SIZE+((3,)))
        encoder_input_label = Input((1))
        self.c1 = Conv2D(64, (5, 5), strides=(1, 1), padding='same', input_shape=IMG_SIZE+((3,)))
        Z = self.c1(encoder_input_img)
        Z = LeakyReLU()(Z)
        Z = Dropout(0.3)(Z)

        self.c2 = Conv2D(128, (5, 5), strides=(2, 2), padding='same')
        Z = self.c2(Z)
        Z = LeakyReLU()(Z)
        Z = Dropout(0.3)(Z)

        self.c3 = Conv2D(128, (5, 5), strides=(2, 2), padding='same')
        Z = self.c3(Z)
        Z = LeakyReLU()(Z)
        Z = Dropout(0.3)(Z)
        Z = Flatten()(Z)

        self.latent_embedding = Dense(EMBEDDED_DIM)(Z)
        return Model(encoder_input_img, self.latent_embedding)
    
    def create_decoder(self):
        
        I = Input(self.latent_embedding.shape)
        Z = Dense(16*16*128)(I)
        Z = Reshape((16,16,128))(Z)


        c3T = Conv2DTransposeTied(c3)(Z)
        c3Ta = LeakyReLU()(c3T)
        c3Td = Dropout(0.3)(c3Ta)

        c2T = Conv2DTransposeTied(c2)(c3Td)
        c2Ta = LeakyReLU()(c2T)
        c2Td = Dropout(0.3)(c2Ta)

        c1T = Conv2DTransposeTied(c1)(c2T)
        c1Ta = LeakyReLU()(c1T)
        c1Td = Dropout(0.3)(c1Ta)

        decoder_output = c1Td
        return Model(I, decoder_output)
        
    def call(self, inputs):
        return self.autoencoder(inputs)

Autoencoder()

In [ ]:
optimizer=tf.keras.optimizers.Adam(lr=.001)

autoencoder =Autoencoder()

autoencoder.compile(optimizer=optimizer, loss='mse')

autoencoder.fit(ae_dataset, epochs=100, steps_per_epoch=100)

In [ ]:
time = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
Autoencoder.save('./autoencoder'+time)

In [ ]:
predictions = autoencoder.predict(train_dataset.take(1))
fig = plt.figure(figsize=(8,4))

for i in range(predictions.shape[0]):
    plt.subplot(4, 8, i+1)
    plt.imshow(((predictions[i, :, :, :]*127.5)+127.5)/255.)
    plt.axis('off')

In [ ]:
rf_dataset = train_dataset.map(lambda x,y : ((x,y),y))
encoder = keras.Model([encoder_input_img,encoder_input_label],[encoder_output_img, encoder_input_label])
print(rf_dataset)
rf_dataset = encoder.predict(rf_dataset, steps=1000) 

In [ ]:
rf_dataset[0].shape
rf_dataset[1].shape

In [ ]:
rf_image_dataset = tf.data.Dataset.from_tensor_slices(rf_dataset[0])
rf_label_dataset = tf.data.Dataset.from_tensor_slices(rf_dataset[1])
rf_combined_dataset = tf.data.Dataset.zip(((rf_image_dataset),(rf_label_dataset)))

In [ ]:
rf_combined_dataset

In [ ]:
import tensorflow_decision_forests as tfdf
from tensorflow.keras import Sequential
import keras

#run encoder to get embeddings
# (encoder_output)
rf_model = tfdf.keras.RandomForestModel()
rf_model.fit(rf_dataset[0],rf_dataset[1])

In [ ]:
def run_inference(rf_model, encoder, input):
    embeddings = encoder.predict(input,tf.ones(input.shape[0]))
    return rf_model.predict(embeddings)
run_inference(rf_model, encoder, rf_dataset[0])